<a href="https://colab.research.google.com/github/amirsakib16/Computer_Graphics/blob/main/EnemyShooterGame_OpenGL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
import math
import random
from OpenGL.GLUT import glutBitmapCharacter, GLUT_BITMAP_HELVETICA_18

camera_pos = [0, 500, 500]
cameraDegree = 0
cameraZpos = 500
collisionStable = 0
COOLDOWN_FRAMES = 30
fovY = 120
GRID_LENGTH = 600
gun_pos = [0, 0]
gunDegree = 0
gunvelocity = 10
gunTorque = 5
bullets = []
bulletsVelocity = 20
enemies = []
MAX_ENEMIES = 5
cheatActive = False
cheatCameraView = False
firstPerson = False
life = 5
bulletsMissed = 0
max_bulletsMissed = 10
game_over = False
score = 0
rand_var = 423

def restart():
    global life
    global bulletsMissed
    global score
    global game_over
    global gun_pos
    global gunDegree
    global bullets
    global enemies
    life = 5
    bulletsMissed = 0
    score = 0
    game_over = False
    gun_pos = [0, 0]
    gunDegree = 0
    bullets.clear()
    enemies.clear()
    for i in range(MAX_ENEMIES):
        newEnemy()
    print(f"Remaining Player Life: {life}")

def newEnemy():
    side = random.randint(1, 4)
    if side == 1:
        x = -random.uniform(-GRID_LENGTH, GRID_LENGTH)
        y = random.uniform(-GRID_LENGTH, GRID_LENGTH)
    elif side == 2:
        x = random.uniform(-GRID_LENGTH, GRID_LENGTH)
        y = random.uniform(-GRID_LENGTH, GRID_LENGTH)
    elif side == 3:
        x = random.uniform(-GRID_LENGTH, GRID_LENGTH)
        y = -random.uniform(-GRID_LENGTH, GRID_LENGTH)
    else:
        x = random.uniform(-GRID_LENGTH, GRID_LENGTH)
        y = random.uniform(-GRID_LENGTH, GRID_LENGTH)
    enemies.append({
        'pos': [x, y],
        'expand': True,
        'radius': 20,
        'scale': 1.0,
        'last_targeted': False,
        'targeted': False})

def draw_text(x, y, text, font=None):
    if font is None:
        font = GLUT_BITMAP_HELVETICA_18
    glColor3f(1, 1, 1)
    glMatrixMode(GL_PROJECTION)
    glPushMatrix()
    glLoadIdentity()
    gluOrtho2D(0, 1000, 0, 800)
    glMatrixMode(GL_MODELVIEW)
    glPushMatrix()
    glLoadIdentity()
    glRasterPos2f(x, y)
    for ch in text:
        glutBitmapCharacter(font, ord(ch))
    glPopMatrix()
    glMatrixMode(GL_PROJECTION)
    glPopMatrix()
    glMatrixMode(GL_MODELVIEW)

def playGround(totalTiles, tileSize):
    for i in range(totalTiles):
        for j in range(totalTiles):
            x_start = -GRID_LENGTH + i * tileSize
            x_end = x_start + tileSize
            y_start = -GRID_LENGTH + j * tileSize
            y_end = y_start + tileSize
            tile_color = (i + j) % 2
            if tile_color == 0:
                glColor3f(1.0, 1.0, 1.0)
            else:
                glColor3f(0.7, 0.5, 0.95)
            glBegin(GL_QUADS)
            glVertex3f(x_start, y_start, 0)
            glVertex3f(x_end, y_start, 0)
            glVertex3f(x_end, y_end, 0)
            glVertex3f(x_start, y_end, 0)
            glEnd()

def boundaries():
    height = 80
    thickness = 10
    glPushMatrix()
    glColor3f(0.0, 1.0, 0.0)
    glTranslatef(-GRID_LENGTH, 0, height / 2)
    glScalef(thickness, GRID_LENGTH * 2, height)
    glutSolidCube(1)
    glPopMatrix()

    glPushMatrix()
    glColor3f(0.0, 0.0, 1.0)
    glTranslatef(GRID_LENGTH, 0, height / 2)
    glScalef(thickness, GRID_LENGTH * 2, height)
    glutSolidCube(1)
    glPopMatrix()

    glPushMatrix()
    glColor3f(0.0, 1.0, 1.0)
    glTranslatef(0, -GRID_LENGTH, height / 2)
    glScalef(GRID_LENGTH * 2, thickness, height)
    glutSolidCube(1)
    glPopMatrix()

    glPushMatrix()
    glColor3f(1.0, 1.0, 1.0)
    glTranslatef(0, GRID_LENGTH, height / 2)
    glScalef(GRID_LENGTH * 2, thickness, height)
    glutSolidCube(1)
    glPopMatrix()


def PLAYER():
    glPushMatrix()
    x, y = gun_pos
    glTranslatef(x, y, 0)
    if game_over:
        glRotatef(-90, 1, 0, 0)
        glTranslatef(0, 0, -20)
    else:
        glRotatef(gunDegree + 90, 0, 0, 1)
    glPushMatrix()
    glColor3f(0, 0.5, 0)
    glTranslatef(0, 0, 55)
    glScalef(35, 15, 70)
    glutSolidCube(1)
    glPopMatrix()

    glPushMatrix()
    glColor3f(0, 0, 0)
    glTranslatef(0, 0, 100)
    glutSolidSphere(15, 20, 20)
    glPopMatrix()

    glPushMatrix()
    glColor3f(0, 0, 1)
    glTranslatef(-13, 0, 10)
    gluCylinder(gluNewQuadric(), 6, 10, 35, 15, 15)
    glPopMatrix()

    glPushMatrix()
    glColor3f(0, 0, 1)
    glTranslatef(13, 0, 10)
    gluCylinder(gluNewQuadric(), 6, 10, 35, 15, 15)
    glPopMatrix()

    glPushMatrix()
    glColor3f(0.93, 0.85, 0.7)
    glTranslatef(-18, 0, 89)
    glRotatef(90, 1, 0, 0)
    gluCylinder(gluNewQuadric(), 4, 7, 38, 10, 10)
    glPopMatrix()

    glPushMatrix()
    glColor3f(0.93, 0.85, 0.7)
    glTranslatef(18, 0, 89)
    glRotatef(90, 1, 0, 0)
    gluCylinder(gluNewQuadric(), 4, 7, 38, 10, 10)
    glPopMatrix()

    glPushMatrix()
    glColor3f(0.6, 0.6, 0.6)
    glTranslatef(0, 0, 90)
    glRotatef(90, 1, 0, 0)
    gluCylinder(gluNewQuadric(), 2.5, 5.5, 50, 12, 8)
    glPopMatrix()

    glPopMatrix()

def ENEMY(enemy):
    glPushMatrix()
    x, y = enemy['pos']
    scale = enemy['scale']
    glTranslatef(x, y, 20 * scale)
    glColor3f(1, 0, 0)
    glutSolidSphere(20 * scale, 20, 20)
    glTranslatef(0, 0, 30 * scale)
    glColor3f(0, 0, 0)
    glutSolidSphere(10 * scale, 20, 20)

    glPopMatrix()

def BULLETS(bullet):
    glPushMatrix()
    x, y, z = bullet['pos']
    glTranslatef(x, y, z)
    glColor3f(1, 0, 0)
    glutSolidCube(8)
    glPopMatrix()

def enemyActivities(enemy):
    speed = 0.1
    px, py = gun_pos
    ex, ey = enemy['pos']
    dx = px - ex
    dy = py - ey
    dist = math.sqrt(dx * dx + dy * dy)
    if dist > 1.0:
        ex += speed * dx / dist
        ey += speed * dy / dist
    enemy['pos'] = [ex, ey]
    scale_change_rate = 0.005
    if enemy['expand']:
        enemy['scale'] += scale_change_rate
        if enemy['scale'] >= 1.2:
            enemy['expand'] = False
    else:
        enemy['scale'] -= scale_change_rate
        if enemy['scale'] <= 0.8:
            enemy['expand'] = True


def detectCollision():
    global bullets
    global enemies
    global score
    global bulletsMissed
    new_bullets = []
    for bullet in bullets:
        bx, by, bz = bullet['pos']
        hit_enemy = False
        for enemy in enemies:
            ex, ey = enemy['pos']
            dist = math.sqrt((bx - ex) ** 2 + (by - ey) ** 2)
            if dist <= 20:
                # Hit detected
                hit_enemy = True
                score += 1
                enemies.remove(enemy)
                newEnemy()
                break
        if not hit_enemy:
            if abs(bx) <= GRID_LENGTH and abs(by) <= GRID_LENGTH:
                new_bullets.append(bullet)
            else:
                if not cheatActive:
                    bulletsMissed += 1
                    print(f"Bullet missed: {bulletsMissed}")
    bullets[:] = new_bullets

def bulletFunctionality():
    for bullet in bullets:
        dx = bullet['dx']
        dy = bullet['dy']
        bullet['pos'][0] += dx * bulletsVelocity
        bullet['pos'][1] += dy * bulletsVelocity

def keyboardListener(key, x, y):
    global gun_pos
    global gunDegree
    global cheatActive
    global cheatCameraView
    global life
    global bulletsMissed
    global bullets
    global game_over
    if game_over:
        if key == b'r':
            restart()
        return
    if key == b'w':
        rad = math.radians(gunDegree)
        gun_pos[0] += gunvelocity * math.cos(rad)
        gun_pos[1] += gunvelocity * math.sin(rad)
    elif key == b's':
        rad = math.radians(gunDegree)
        gun_pos[0] -= gunvelocity * math.cos(rad)
        gun_pos[1] -= gunvelocity * math.sin(rad)
    elif key == b'a':
        gunDegree += gunTorque
        if gunDegree >= 360:
            gunDegree -= 360
    elif key == b'd':
        gunDegree -= gunTorque
        if gunDegree < 0:
            gunDegree += 360
    elif key == b'c':
        cheatActive = not cheatActive
        if not cheatActive:
            bulletsMissed = bulletsMissed
            bullets.clear()
    elif key == b'v':
        cheatCameraView = not cheatCameraView
    elif key == b'r':
        restart()

def specialKeyListener(key, x, y):
    global camera_pos
    global cameraDegree
    global cameraZpos
    cx, cy, cz = camera_pos
    if key == GLUT_KEY_UP:
        cameraZpos += 20
        if cameraZpos > 2000:
            cameraZpos = 2000
    elif key == GLUT_KEY_DOWN:
        cameraZpos -= 20
        if cameraZpos < 100:
            cameraZpos = 100
    elif key == GLUT_KEY_LEFT:
        cameraDegree -= 5
        if cameraDegree < 0:
            cameraDegree += 360
    elif key == GLUT_KEY_RIGHT:
        cameraDegree += 5
        if cameraDegree >= 360:
            cameraDegree -= 360
    rad = math.radians(cameraDegree)
    distance = 500
    camera_pos[0] = distance * math.cos(rad)
    camera_pos[1] = distance * math.sin(rad)
    camera_pos[2] = cameraZpos

def mouseListener(button, state, x, y):
    global bullets
    global firstPerson
    global gun_pos
    global gunDegree
    if state != GLUT_DOWN:
        return
    if button == GLUT_LEFT_BUTTON:
        if game_over:
            return
        rad = math.radians(gunDegree)
        dx = math.cos(rad)
        dy = math.sin(rad)
        perp_dx = -dy
        perp_dy = dx
        forward_offset = 50
        side_offset = 3
        bullet_x = gun_pos[0] + dx * forward_offset + perp_dx * side_offset
        bullet_y = gun_pos[1] + dy * forward_offset + perp_dy * side_offset
        bullet_z = 90
        bullets.append({'pos': [bullet_x, bullet_y, bullet_z], 'dx': dx, 'dy': dy})
        print("Player Bullet Fired!")
    elif button == GLUT_RIGHT_BUTTON:
        firstPerson = not firstPerson

isCheatOn = False
def setupCamera():
    global isCheatOn
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(fovY, 1.25, 0.1, 1500)
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    if firstPerson:
        if cheatCameraView:
            isCheatOn = True
            rad = math.radians(gunDegree)
            observerX = gun_pos[0] - 1 * math.cos(rad)
            observerY = gun_pos[1]  - 1 * math.sin(rad)
            observerZ = 120
            cameraPositionX = gun_pos[0] + 70 * math.cos(rad)
            cameraPositionY = gun_pos[1] + 70 * math.sin(rad)
            cameraPositionZ = 120
            gluLookAt(observerX, observerY, observerZ, cameraPositionX, cameraPositionY, cameraPositionZ, 0, 0, 1)
        else:
            rad = math.radians(gunDegree)
            observerX = gun_pos[0] - 1 * math.cos(rad)
            observerY = gun_pos[1]  - 1 * math.sin(rad)
            observerZ = 120
            if not isCheatOn:
                cameraPositionX = gun_pos[0]
                cameraPositionY = gun_pos[1]
            else:
                cameraPositionX = gun_pos[0]
                cameraPositionY = gun_pos[1] + 5
            cameraPositionZ = 120
            gluLookAt(observerX, observerY, observerZ, cameraPositionX, cameraPositionY, cameraPositionZ, 0, 0, 1)
    else:
        cx, cy, cz = camera_pos
        gluLookAt(cx, cy, cz, 0, 0, 0, 0, 0, 1)

def isCollision():
    global life
    global game_over
    global enemies
    global collisionStable
    if game_over:
        return
    if collisionStable > 0:
        collisionStable -= 1
        return
    px, py = gun_pos
    for i, enemy in enumerate(enemies):
        ex, ey = enemy['pos']
        dist = math.sqrt((px - ex) ** 2 + (py - ey) ** 2)
        if dist < 40:
            life -= 1
            print(f"Remaining Player Life: {life}")
            collisionStable = COOLDOWN_FRAMES
            del enemies[i]
            newEnemy()
            if life <= 0:
                life = 0
                game_over = True
                enemies.clear()
            break

def automatedKill():
    global bullets
    global gunDegree
    gunDegree += 5
    if gunDegree >= 360:
        gunDegree -= 360
    px, py = gun_pos
    tolerance = 15
    closest_enemy = None
    min_dist = float('inf')
    rad = math.radians(gunDegree)
    gun_dir = (math.cos(rad), math.sin(rad))
    for enemy in enemies:
        ex, ey = enemy['pos']
        dx = ex - px
        dy = ey - py
        dist = math.sqrt(dx*dx + dy*dy)
        if dist == 0:
            continue
        dir_to_enemy = (dx / dist, dy / dist)
        dot = gun_dir[0] * dir_to_enemy[0] + gun_dir[1] * dir_to_enemy[1]
        angle_between = math.degrees(math.acos(dot))
        if angle_between <= tolerance:
            if dist < min_dist:
                min_dist = dist
                closest_enemy = enemy
    if closest_enemy:
        ex, ey = closest_enemy['pos']
        dx = ex - px
        dy = ey - py
        dist = math.sqrt(dx*dx + dy*dy)
        dir_to_enemy = (dx / dist, dy / dist)
        bullet_x = px + dir_to_enemy[0] * 30
        bullet_y = py + dir_to_enemy[1] * 30
        bullet_z = 60
        bullets.append({'pos': [bullet_x, bullet_y, bullet_z],'dx': dir_to_enemy[0], 'dy': dir_to_enemy[1]})

def idle():
    global life
    global bulletsMissed
    global game_over
    if game_over:
        glutPostRedisplay()
        return
    if cheatActive:
        automatedKill()
    bulletFunctionality()
    for enemy in enemies:
        enemyActivities(enemy)
    detectCollision()
    isCollision()
    if bulletsMissed >= max_bulletsMissed:
        game_over = True
        enemies.clear()
    glutPostRedisplay()

def showScreen():
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glViewport(0, 0, 1000, 800)
    glLoadIdentity()
    setupCamera()
    totalTiles = (GRID_LENGTH * 2) // 50
    playGround(totalTiles, 50)
    boundaries()
    for enemy in enemies:
        ENEMY(enemy)
    for bullet in bullets:
        BULLETS(bullet)
    PLAYER()
    if game_over:
        draw_text(10, 750, f"Game is Over. Your Score is {score}")
        draw_text(10, 725, f'Press "R" to RESTART the Game.')
    else:
        draw_text(10, 750, f"Player Life Remaining: {life}")
        draw_text(10, 725, f"Game Score: {score}")
        draw_text(10, 700, f"Player Bullet Missed: {bulletsMissed}")

    glutSwapBuffers()

def main():
    glutInit()
    glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB | GLUT_DEPTH)
    glutInitWindowSize(1000, 800)
    glutInitWindowPosition(0, 0)
    wind = glutCreateWindow(b"3D OpenGL Intro")
    glClearColor(0, 0, 0, 1)
    glEnable(GL_DEPTH_TEST)
    restart()
    glutDisplayFunc(showScreen)
    glutKeyboardFunc(keyboardListener)
    glutSpecialFunc(specialKeyListener)
    glutMouseFunc(mouseListener)
    glutIdleFunc(idle)

    glutMainLoop()


if __name__ == "__main__":
    main()
